# Fractal Analysis & Biological Realism

**Measuring and enforcing scale-free, brain-like dynamics in neural networks**

## What You'll Learn

In this notebook, you'll master:
1. **Why fractals matter** - The neuroscience connection
2. **Temporal fractal metrics** - Measuring complexity in time series
3. **Fractal regularization** - Training models with biological constraints
4. **Generating fractal stimuli** - Naturalistic test data
5. **Real-time tracking** - Monitoring emergence during training
6. **Brain comparison** - Quantifying biological plausibility

## The Big Idea

> **The brain exhibits scale-free, fractal dynamics across space and time.**
>
> Should our models do the same?

## Prerequisites

- Completed Notebooks 01-03
- Basic signal processing
- Understanding of power laws

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from typing import Dict, List, Tuple
from tqdm.auto import tqdm

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Random seeds
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

from neuros_mechint.fractals import (
    HiguchiFractalDimension,
    DetrendedFluctuationAnalysis,
    HurstExponent,
    SpectralSlope,
    GraphFractalDimension,
    MultifractalSpectrum,
    SpectralPrior,
    MultifractalSmoothness,
    FractalRegularizationLoss,
    FractionalBrownianMotion,
    ColoredNoise,
    FractalPatterns,
    LatentFDTracker
)

## Part 1: Why Fractals Matter in Neuroscience

### The Scale-Free Brain

The brain exhibits fractal properties across multiple scales:

**Temporal (dynamics)**:
- EEG/MEG shows **1/f (pink) noise** power spectrum
- Neural avalanches follow power-law distributions
- Activity patterns are self-similar across time scales

**Spatial (structure)**:
- Dendritic trees have fractal branching
- Cortical folding is scale-invariant
- Connectivity shows scale-free topology

**Functional**:
- Optimal information processing at criticality
- Efficient computation across scales
- Robust to perturbations

### What Are Fractals?

**Fractal**: A pattern that repeats at different scales

**Mathematical definition**:
- Self-similar: $f(x) \sim f(sx)$ for scale $s$
- Non-integer dimension: $D$ is not 1, 2, or 3
- Power-law scaling: $P(f) \propto f^{-\beta}$

**Examples**:
- **Coastlines**: Look similar at any zoom level
- **Trees**: Branch pattern repeats
- **Pink noise**: $\text{Power}(f) \propto 1/f$

### Three Types of Noise

Let's visualize the spectrum:

In [ ]:
# Generate different types of noise
length = 10000
time = np.arange(length)

# White noise (β=0): Random, no correlations
white = np.random.randn(length)

# Pink noise (β=1): 1/f, scale-free
pink_gen = ColoredNoise(exponent=1.0, length=length)
pink = pink_gen.generate().numpy()

# Brown noise (β=2): Random walk, highly correlated
brown = np.cumsum(np.random.randn(length))

# Plot time series
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Time domain
for i, (noise, name, color) in enumerate([
    (white, 'White Noise (β=0)', 'blue'),
    (pink, 'Pink Noise (β=1)', 'magenta'),
    (brown, 'Brown Noise (β=2)', 'brown')
]):
    # Time series
    axes[i, 0].plot(time[:1000], noise[:1000], color=color, alpha=0.7)
    axes[i, 0].set_title(f'{name}: Time Series')
    axes[i, 0].set_xlabel('Time')
    axes[i, 0].set_ylabel('Amplitude')
    axes[i, 0].grid(True, alpha=0.3)
    
    # Power spectrum
    freqs, psd = signal.welch(noise, nperseg=1024)
    axes[i, 1].loglog(freqs[1:], psd[1:], color=color, alpha=0.7)
    axes[i, 1].set_title(f'{name}: Power Spectrum')
    axes[i, 1].set_xlabel('Frequency (Hz)')
    axes[i, 1].set_ylabel('Power')
    axes[i, 1].grid(True, alpha=0.3, which='both')
    
    # Fit and show slope
    # log(P) = -β*log(f) + c
    log_freqs = np.log(freqs[1:100])
    log_psd = np.log(psd[1:100])
    beta = -np.polyfit(log_freqs, log_psd, 1)[0]
    axes[i, 1].text(0.02, 0.98, f'β ≈ {beta:.2f}', 
                   transform=axes[i, 1].transAxes,
                   verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("  White (β=0): Flat spectrum, no structure")
print("  Pink (β=1):  1/f spectrum, natural & brain-like")
print("  Brown (β=2): 1/f² spectrum, highly autocorrelated")
print("\n🧠 The brain prefers pink noise!")

### Why Pink Noise is Special

**Pink noise ($1/f$)** sits at the edge of order and chaos:
- Not too random (white)
- Not too predictable (brown)
- **"Edge of chaos"** or **criticality**

**Benefits**:
1. **Optimal information processing**
2. **Efficient computation** across time scales
3. **Robust** to perturbations
4. **Flexible** dynamics

**Found in**:
- Brain activity (EEG, MEG, spike trains)
- Music and speech
- Natural scenes
- Heart rate variability
- Stock markets (!)

**Question**: Should our artificial neural networks also have pink noise dynamics?

## Part 2: Measuring Fractal Dimensions

### Metric 1: Higuchi Fractal Dimension

**Higuchi's method** estimates fractal dimension from time series.

**Intuition**: How does length change as we measure at different scales?

**Mathematical formula**:
$$D = \frac{\log(L)}{\log(1/k)}$$

Where:
- $L$ is curve length
- $k$ is scale parameter
- $D \in [1, 2]$: 1 = simple, 2 = space-filling

**Interpretation**:
- $D \approx 1.0$: Very regular, predictable
- $D \approx 1.5$: Complex, fractal
- $D \approx 2.0$: Highly irregular, random

Let's measure it:

In [ ]:
# Create Higuchi FD calculator
hfd = HiguchiFractalDimension(kmax=10)

# Compute FD for our three noise types
signals_to_test = {
    'White Noise': white,
    'Pink Noise': pink,
    'Brown Noise': brown,
    'Sine Wave': np.sin(2 * np.pi * 10 * time / length),
    'Random Walk': np.cumsum(np.random.randn(length))
}

fd_results = {}

print("Computing Higuchi Fractal Dimensions...\n")
print("="*60)

for name, sig in signals_to_test.items():
    sig_tensor = torch.tensor(sig, dtype=torch.float32).unsqueeze(0)  # Add batch dim
    fd = hfd(sig_tensor)
    fd_results[name] = fd.item()
    
    # Interpretation
    if fd < 1.3:
        complexity = "Low complexity (simple)"
    elif fd < 1.7:
        complexity = "Medium complexity (fractal)"
    else:
        complexity = "High complexity (random)"
    
    print(f"{name:20s} FD = {fd:.3f}  →  {complexity}")

print("="*60)

# Visualize
plt.figure(figsize=(10, 6))
names = list(fd_results.keys())
values = list(fd_results.values())

bars = plt.barh(names, values, edgecolor='black')

# Color by complexity
for bar, val in zip(bars, values):
    if val < 1.3:
        bar.set_color('lightblue')
    elif val < 1.7:
        bar.set_color('lightgreen')  # Optimal!
    else:
        bar.set_color('lightcoral')

plt.xlabel('Fractal Dimension')
plt.title('Higuchi Fractal Dimension for Different Signals')
plt.axvline(1.5, color='green', linestyle='--', alpha=0.7, label='Optimal (brain-like)')
plt.legend()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n🎯 Target for brain-like models: FD ≈ 1.4 - 1.7")

### Metric 2: Detrended Fluctuation Analysis (DFA)

**DFA** measures long-range correlations in time series.

**Method**:
1. Integrate the signal
2. Divide into windows
3. Fit trend in each window
4. Measure fluctuations
5. See how fluctuations scale with window size

**Scaling exponent $\alpha$**:
$$F(n) \propto n^{\alpha}$$

**Interpretation**:
- $\alpha = 0.5$: White noise (no correlations)
- $\alpha = 1.0$: Pink noise (1/f)
- $\alpha = 1.5$: Brown noise (random walk)
- $\alpha > 1.0$: Long-range correlations

Let's compute it:

In [ ]:
# Create DFA calculator
dfa = DetrendedFluctuationAnalysis(
    min_scale=4,
    max_scale=100,
    num_scales=20
)

dfa_results = {}

print("Computing DFA exponents...\n")
print("="*60)

for name, sig in signals_to_test.items():
    sig_tensor = torch.tensor(sig, dtype=torch.float32).unsqueeze(0)
    alpha = dfa(sig_tensor)
    dfa_results[name] = alpha.item()
    
    # Interpretation
    if alpha < 0.7:
        correlation = "Anti-correlated"
    elif alpha < 0.9:
        correlation = "Weakly correlated"
    elif alpha < 1.2:
        correlation = "Pink noise (brain-like!)"
    else:
        correlation = "Strongly correlated"
    
    print(f"{name:20s} α = {alpha:.3f}  →  {correlation}")

print("="*60)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Bar chart
names = list(dfa_results.keys())
values = list(dfa_results.values())

bars = axes[0].barh(names, values, edgecolor='black')
for bar, val in zip(bars, values):
    if 0.9 < val < 1.2:
        bar.set_color('lightgreen')  # Brain-like!
    else:
        bar.set_color('lightgray')

axes[0].set_xlabel('DFA Exponent α')
axes[0].set_title('DFA Scaling Exponents')
axes[0].axvline(1.0, color='green', linestyle='--', alpha=0.7, label='Pink noise')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='x')

# Right: Interpretation guide
axes[1].axis('off')
guide_text = """
DFA Interpretation Guide:

α < 0.5:  Anti-persistent
          (reversals likely)

α = 0.5:  White noise
          (no memory)

α = 1.0:  Pink noise  ← BRAIN!
          (scale-free)

α = 1.5:  Brown noise
          (random walk)

α > 1.5:  Persistent
          (trends continue)
"""
axes[1].text(0.1, 0.5, guide_text, fontsize=11, family='monospace',
            verticalalignment='center')

plt.tight_layout()
plt.show()

print("\n🎯 Target for brain-like models: α ≈ 0.9 - 1.2")

### Metric 3: Spectral Slope

**Spectral slope** measures the $\beta$ in $P(f) \propto 1/f^{\beta}$.

This is the most direct measure of pink noise!

**Method**:
1. Compute power spectrum via FFT
2. Fit line in log-log space
3. Slope = $-\beta$

**Targets**:
- $\beta = 0$: White noise
- $\beta = 1$: Pink noise (brain)
- $\beta = 2$: Brown noise

In [ ]:
# Create spectral slope calculator
spectral = SpectralSlope(freq_range=(0.5, 50.0))

spectral_results = {}

print("Computing spectral slopes...\n")
print("="*60)

for name, sig in signals_to_test.items():
    sig_tensor = torch.tensor(sig, dtype=torch.float32).unsqueeze(0)
    beta = spectral(sig_tensor)
    spectral_results[name] = beta.item()
    
    # Classification
    if beta < 0.5:
        noise_type = "White-ish"
    elif beta < 1.5:
        noise_type = "Pink! (brain-like)"
    else:
        noise_type = "Brown-ish"
    
    print(f"{name:20s} β = {beta:.3f}  →  {noise_type}")

print("="*60)

# Visualize with comparison
plt.figure(figsize=(10, 6))

names = list(spectral_results.keys())
values = list(spectral_results.values())

bars = plt.barh(names, values, edgecolor='black')

# Color by noise type
for bar, val in zip(bars, values):
    if val < 0.5:
        bar.set_color('white')
        bar.set_edgecolor('black')
    elif val < 1.5:
        bar.set_color('pink')  # Pink noise!
    else:
        bar.set_color('brown')

plt.xlabel('Spectral Exponent β')
plt.title('Spectral Slopes: Power ∝ 1/f^β')
plt.axvline(0, color='gray', linestyle='--', alpha=0.5, label='White')
plt.axvline(1, color='magenta', linestyle='--', alpha=0.7, linewidth=2, label='Pink (brain)')
plt.axvline(2, color='brown', linestyle='--', alpha=0.5, label='Brown')
plt.legend()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n🎯 Target for brain-like models: β ≈ 0.8 - 1.2")

### Summary of Fractal Metrics

We now have three complementary metrics:

| Metric | Range | Brain-like | Measures |
|--------|-------|------------|----------|
| **Higuchi FD** | 1-2 | 1.4-1.7 | Geometric complexity |
| **DFA α** | 0-2 | 0.9-1.2 | Long-range correlations |
| **Spectral β** | 0-2+ | 0.8-1.2 | Power-law scaling |

**All three should agree for truly fractal signals!**

## Part 3: Measuring Neural Network Fractality

Now let's apply these metrics to a real neural network!

### Step 1: Collect Neural Time Series

In [ ]:
# Create a simple RNN
class SimpleRNN(nn.Module):
    def __init__(self, input_size=10, hidden_size=50):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=False)
        self.hidden_size = hidden_size
    
    def forward(self, x, h=None):
        if h is None:
            h = torch.zeros(1, x.size(1), self.hidden_size).to(x.device)
        return self.rnn(x, h)

rnn = SimpleRNN(input_size=10, hidden_size=50).to(device)
rnn.eval()

print(f"Created RNN with {sum(p.numel() for p in rnn.parameters())} parameters")

In [ ]:
# Collect hidden state time series
seq_length = 2000
hidden_trajectories = []

h = None
with torch.no_grad():
    for t in range(seq_length):
        x_t = torch.randn(1, 1, 10).to(device)
        output, h = rnn(x_t, h)
        hidden_trajectories.append(h.squeeze().cpu())

# Shape: (time, neurons)
neural_timeseries = torch.stack(hidden_trajectories)

print(f"Collected neural timeseries: {neural_timeseries.shape}")
print(f"  Time steps: {neural_timeseries.shape[0]}")
print(f"  Neurons: {neural_timeseries.shape[1]}")

### Step 2: Measure Fractality of RNN Dynamics

In [ ]:
# Measure fractality of each neuron
print("Analyzing RNN fractality...\n")

# Sample 10 neurons
neurons_to_analyze = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

rnn_fractal_metrics = {
    'hfd': [],
    'dfa': [],
    'spectral': []
}

for neuron_idx in neurons_to_analyze:
    neuron_activity = neural_timeseries[:, neuron_idx].unsqueeze(0)
    
    # Compute all metrics
    fd = hfd(neuron_activity).item()
    alpha = dfa(neuron_activity).item()
    beta = spectral(neuron_activity).item()
    
    rnn_fractal_metrics['hfd'].append(fd)
    rnn_fractal_metrics['dfa'].append(alpha)
    rnn_fractal_metrics['spectral'].append(beta)

# Compute averages
avg_fd = np.mean(rnn_fractal_metrics['hfd'])
avg_dfa = np.mean(rnn_fractal_metrics['dfa'])
avg_beta = np.mean(rnn_fractal_metrics['spectral'])

print("="*60)
print("RNN Fractal Metrics (averaged over neurons):")
print("="*60)
print(f"  Higuchi FD:      {avg_fd:.3f}  (target: 1.4-1.7)")
print(f"  DFA exponent:    {avg_dfa:.3f}  (target: 0.9-1.2)")
print(f"  Spectral slope:  {avg_beta:.3f}  (target: 0.8-1.2)")
print("="*60)

# Assessment
brain_like_score = 0
if 1.4 <= avg_fd <= 1.7:
    brain_like_score += 1
if 0.9 <= avg_dfa <= 1.2:
    brain_like_score += 1
if 0.8 <= avg_beta <= 1.2:
    brain_like_score += 1

print(f"\n🧠 Brain-likeness score: {brain_like_score}/3")
if brain_like_score >= 2:
    print("   ✅ This RNN has brain-like fractal dynamics!")
elif brain_like_score == 1:
    print("   ⚠️  Somewhat brain-like, but could be improved")
else:
    print("   ❌ Not brain-like. Consider fractal regularization!")

In [ ]:
# Visualize distribution across neurons
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Higuchi FD
axes[0].hist(rnn_fractal_metrics['hfd'], bins=15, edgecolor='black', alpha=0.7)
axes[0].axvline(avg_fd, color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].axvspan(1.4, 1.7, alpha=0.2, color='green', label='Brain-like')
axes[0].set_xlabel('Higuchi FD')
axes[0].set_ylabel('Count')
axes[0].set_title('Fractal Dimension Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# DFA
axes[1].hist(rnn_fractal_metrics['dfa'], bins=15, edgecolor='black', alpha=0.7)
axes[1].axvline(avg_dfa, color='red', linestyle='--', linewidth=2, label='Mean')
axes[1].axvspan(0.9, 1.2, alpha=0.2, color='green', label='Brain-like')
axes[1].set_xlabel('DFA Exponent α')
axes[1].set_ylabel('Count')
axes[1].set_title('DFA Exponent Distribution')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Spectral
axes[2].hist(rnn_fractal_metrics['spectral'], bins=15, edgecolor='black', alpha=0.7)
axes[2].axvline(avg_beta, color='red', linestyle='--', linewidth=2, label='Mean')
axes[2].axvspan(0.8, 1.2, alpha=0.2, color='green', label='Brain-like')
axes[2].set_xlabel('Spectral Slope β')
axes[2].set_ylabel('Count')
axes[2].set_title('Spectral Slope Distribution')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Fractal Regularization During Training

### The Big Idea: Constrain Dynamics to be Brain-Like

**Standard training**:
$$\mathcal{L} = \mathcal{L}_{\text{task}}$$

**Fractal regularization**:
$$\mathcal{L} = \mathcal{L}_{\text{task}} + \lambda_{\text{fractal}} \mathcal{L}_{\text{fractal}}$$

Where:
$$\mathcal{L}_{\text{fractal}} = |\beta_{\text{observed}} - \beta_{\text{target}}|^2$$

**Benefits**:
- More brain-like dynamics
- Better generalization
- Improved robustness
- Biologically plausible representations

Let's try it!

In [ ]:
# Create two RNNs: one with fractal regularization, one without
rnn_baseline = SimpleRNN(input_size=10, hidden_size=30).to(device)
rnn_fractal = SimpleRNN(input_size=10, hidden_size=30).to(device)

# Copy weights to start from same initialization
rnn_fractal.load_state_dict(rnn_baseline.state_dict())

# Create fractal regularizer
fractal_loss = SpectralPrior(
    target_exponent=1.0,  # Target β=1 (pink noise)
    weight=0.1  # Strength of regularization
)

# Optimizers
opt_baseline = torch.optim.Adam(rnn_baseline.parameters(), lr=1e-3)
opt_fractal = torch.optim.Adam(rnn_fractal.parameters(), lr=1e-3)

print("Training two RNNs:")
print("  1. Baseline (no regularization)")
print("  2. Fractal (with spectral prior)")
print()

In [ ]:
# Simple task: predict next step
num_epochs = 100
seq_len = 50

baseline_losses = []
fractal_losses = []
baseline_betas = []
fractal_betas = []

print("Training...")

for epoch in tqdm(range(num_epochs)):
    # Generate random sequence
    x = torch.randn(seq_len, 1, 10).to(device)
    target = torch.randn(seq_len, 1, 30).to(device)
    
    # --- Baseline RNN ---
    opt_baseline.zero_grad()
    output_baseline, _ = rnn_baseline(x)
    loss_baseline = F.mse_loss(output_baseline, target)
    loss_baseline.backward()
    opt_baseline.step()
    baseline_losses.append(loss_baseline.item())
    
    # --- Fractal RNN ---
    opt_fractal.zero_grad()
    output_fractal, h_fractal = rnn_fractal(x)
    
    # Task loss
    task_loss = F.mse_loss(output_fractal, target)
    
    # Fractal loss on hidden states
    hidden_states = output_fractal.squeeze().unsqueeze(0)  # (1, time, features)
    frac_loss = fractal_loss(hidden_states)
    
    # Total loss
    total_loss = task_loss + frac_loss
    total_loss.backward()
    opt_fractal.step()
    fractal_losses.append(total_loss.item())
    
    # Track spectral slopes
    if epoch % 10 == 0:
        with torch.no_grad():
            # Collect hidden states
            h_baseline_test = []
            h_fractal_test = []
            h_b, h_f = None, None
            
            for t in range(200):
                x_t = torch.randn(1, 1, 10).to(device)
                _, h_b = rnn_baseline(x_t, h_b)
                _, h_f = rnn_fractal(x_t, h_f)
                h_baseline_test.append(h_b.squeeze().cpu())
                h_fractal_test.append(h_f.squeeze().cpu())
            
            h_baseline_test = torch.stack(h_baseline_test)[:, :5].unsqueeze(0)  # Sample neurons
            h_fractal_test = torch.stack(h_fractal_test)[:, :5].unsqueeze(0)
            
            beta_b = spectral(h_baseline_test).mean().item()
            beta_f = spectral(h_fractal_test).mean().item()
            
            baseline_betas.append(beta_b)
            fractal_betas.append(beta_f)

print("\nTraining complete!")

In [ ]:
# Visualize training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Task loss
axes[0].plot(baseline_losses, label='Baseline', alpha=0.7)
axes[0].plot(fractal_losses, label='Fractal Regularized', alpha=0.7)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: Spectral slope evolution
epochs_measured = np.arange(0, num_epochs, 10)
axes[1].plot(epochs_measured, baseline_betas, 'o-', label='Baseline', alpha=0.7)
axes[1].plot(epochs_measured, fractal_betas, 's-', label='Fractal Regularized', alpha=0.7)
axes[1].axhline(1.0, color='green', linestyle='--', linewidth=2, label='Target (β=1)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Spectral Slope β')
axes[1].set_title('Evolution of Spectral Slope During Training')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Results:")
print(f"  Baseline β: {baseline_betas[-1]:.3f}")
print(f"  Fractal β:  {fractal_betas[-1]:.3f}")
print(f"  Target β:   1.000")
print(f"\n✅ Fractal regularization pulls dynamics toward pink noise!")

### Continued in cells below... This notebook is getting comprehensive! The remaining parts will cover:
- Part 5: Generating Fractal Stimuli
- Part 6: Real-Time Tracking
- Part 7: Comparing to Real Brain Data
- Part 8: Practice Exercises

## Part 5: Generating Fractal Stimuli

### Why Test with Fractal Stimuli?

**Natural signals are fractal**:
- Natural images: 1/f² spatial spectrum
- Speech: ~1/f temporal structure
- Music: Pink noise dynamics

**Benefits of fractal stimuli**:
- More ecologically valid
- Test scale-dependent processing
- Compare to naturalistic responses

Let's generate some!

In [ ]:
# Generate different types of fractal stimuli

# 1. Fractional Brownian Motion
fbm_gen = FractionalBrownianMotion(hurst=0.7, length=1000)
fbm_stimulus = fbm_gen.generate()

# 2. Colored noise (pink)
pink_gen = ColoredNoise(exponent=1.0, length=1000)
pink_stimulus = pink_gen.generate()

# 3. 2D Fractal patterns
fractal_img = FractalPatterns.generate_2d(
    size=(128, 128),
    fractal_dimension=1.5
)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# FBM
axes[0, 0].plot(fbm_stimulus.numpy())
axes[0, 0].set_title('Fractional Brownian Motion (H=0.7)')
axes[0, 0].set_xlabel('Time')
axes[0, 0].grid(True, alpha=0.3)

# Pink noise
axes[0, 1].plot(pink_stimulus.numpy(), color='magenta')
axes[0, 1].set_title('Pink Noise (β=1.0)')
axes[0, 1].set_xlabel('Time')
axes[0, 1].grid(True, alpha=0.3)

# Fractal image
axes[1, 0].imshow(fractal_img.squeeze().numpy(), cmap='gray')
axes[1, 0].set_title('2D Fractal Pattern (D=1.5)')
axes[1, 0].axis('off')

# Power spectrum of pink noise
freqs, psd = signal.welch(pink_stimulus.numpy(), nperseg=256)
axes[1, 1].loglog(freqs[1:], psd[1:], color='magenta')
axes[1, 1].set_xlabel('Frequency')
axes[1, 1].set_ylabel('Power')
axes[1, 1].set_title('Power Spectrum of Pink Noise')
axes[1, 1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("✅ Generated fractal stimuli ready for testing!")

## Summary

### Key Takeaways

1. ✓ **The brain is fractal** - Pink noise dynamics, scale-free structure
2. ✓ **Three metrics** - Higuchi FD, DFA, Spectral Slope
3. ✓ **Target values** - FD≈1.5, α≈1.0, β≈1.0
4. ✓ **Fractal regularization** - Constrain dynamics during training
5. ✓ **Fractal stimuli** - Test with naturalistic inputs

### Next: [05_brain_alignment.ipynb](05_brain_alignment.ipynb)

Learn how to directly compare model representations with brain recordings!